Répartition des scores : histogramme (pour chaque mois ?)
Moyenne des scores au fil du temps (pour chaque mois ?), graphique avec des points
Top 3/5 du positif : histogramme horizontal
Top 3/5 du négatif : histogramme horizontal aussi ?
Nuage de mots pertinents
Idées de données : Nombre total d'avis, nombre d'avis sur une période donnée, amélioration de la note au cours du temps

In [81]:
import pandas as pd
import plotly.io as pio
import numpy as np
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
import re


In [82]:
figs_json = []

In [83]:
df = pd.read_csv('processed_data.csv')
df

,Name,grading,date,Text,Target,Language,label (roberta),score (roberta)
0,Wissem Smaali,1,"{'months': 11, 'years': 0}","To avoid\r\nThe Piraeus bakery in Vitry, the s...",0,en,NEGATIVE,0.889870
1,ted tenore,1,"{'months': 4, 'years': 0}",I specify the star I give it only for the croi...,0,en,NEGATIVE,0.558830
2,Sly yo!,1,"{'months': 4, 'years': 0}",Too bad for the pastry chef who struggles and ...,0,en,NEGATIVE,0.916203
3,Emiie.R D,1,"{'months': 1, 'years': 0}",Too bad the products are very good but the sal...,0,en,NEGATIVE,0.777332
4,Michael,1,"{'months': 1, 'years': 0}","If I could put -1000 I would have, I asked for...",0,en,NEUTRAL,0.500744
...,...,...,...,...,...,...,...,...
82,sonia bobrie,5,"{'months': 3, 'years': 0}",Quality service in the dressing room/storage d...,2,en,POSITIVE,0.989260
83,Aleksander Callaud,5,"{'months': 24, 'years': 2}","Very good restaurant, with very friendly staff...",2,en,POSITIVE,0.987408
84,La fée toute seule,5,"{'months': 10, 'years': 0}",A safe bet in Beauvais. Good reception and fai...,2,en,POSITIVE,0.952969
85,nicolas gauthier,5,"{'months': 12, 'years': 1}","Very good burger, and very good draft beer. Go...",2,en,POSITIVE,0.974670


# Mots importants

In [84]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mrics\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mrics\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [85]:
# Convertir la colonne "text" en une liste de tous les commentaires
commentaires = df['Text'].tolist()

# Concaténer tous les commentaires en une seule chaîne de caractères
texte_complet = ' '.join(commentaires)

# Diviser la chaîne de caractères en mots individuels
mots = texte_complet.split()


In [86]:
# Utiliser l'étiquetage morpho-syntaxique pour identifier les noms communs
noms_communs = [mot for mot, pos in nltk.pos_tag(mots) if pos.startswith('NN')]

In [87]:
# Compter les occurrences de chaque nom commun
compteur_noms_communs = Counter(noms_communs)

# Récupérer les trois noms communs les plus fréquents et leurs occurrences
top_noms_communs = compteur_noms_communs.most_common(3)

top_noms_communs


[('bakery', 16), ('time', 11), ('room', 11)]

In [88]:


# Créer une liste de tous les commentaires positifs, négatifs et neutres
commentaires_positifs = df[df['label (roberta)'] == 'POSITIVE']['Text']
commentaires_negatifs = df[df['label (roberta)'] == 'NEGATIVE']['Text']
commentaires_neutres = df[df['label (roberta)'] == 'NEUTRAL']['Text']

for i in top_noms_communs:
    nom = i[0]
    # Compter le nombre de commentaires positifs, négatifs, et neutres pour le nom courant
    nb_positifs = commentaires_positifs.str.contains(nom, flags=re.IGNORECASE, regex=True).sum()
    nb_negatifs = commentaires_negatifs.str.contains(nom, flags=re.IGNORECASE, regex=True).sum()
    nb_neutres = commentaires_neutres.str.contains(nom, flags=re.IGNORECASE, regex=True).sum()

    # Créer une liste contenant le nombre de commentaires positifs, négatifs, et neutres pour le nom courant
    liste = [nb_positifs, nb_negatifs, nb_neutres]

    # Creer une pie chart pour le nom courant
    fig = go.Figure(data=[go.Pie(labels=['Positif', 'Négatif', 'Neutre'], values=liste)])
    fig.update_layout(title_text='Sentiment des commentaires contenant le mot "' + nom + '"')
    fig.show()
    fig_json = fig.to_json()
    figs_json.append(fig_json)



# Subgraph Pourcentage de sentiments positifs, neutres, négatifs au fil du temps (3 courbes, un pour chaque)

In [89]:
df_groupby_date_label = df.groupby(['date', 'label (roberta)']).size().unstack(fill_value=0)
df_groupby_date_label

label (roberta),NEGATIVE,NEUTRAL,POSITIVE
date,,,
"{'months': 1, 'years': 0}",5,1,4
"{'months': 10, 'years': 0}",0,0,2
"{'months': 11, 'years': 0}",3,0,0
"{'months': 12, 'years': 1}",4,1,13
"{'months': 2, 'years': 0}",3,0,2
"{'months': 24, 'years': 2}",3,1,4
"{'months': 3, 'years': 0}",3,1,1
"{'months': 36, 'years': 3}",0,2,2
"{'months': 4, 'years': 0}",3,0,3


In [90]:
df_groupby_date_label['TOTAL'] = df_groupby_date_label[['NEGATIVE', 'NEUTRAL', 'POSITIVE']].sum(axis=1)
df_groupby_date_label

label (roberta),NEGATIVE,NEUTRAL,POSITIVE,TOTAL
date,,,,
"{'months': 1, 'years': 0}",5,1,4,10
"{'months': 10, 'years': 0}",0,0,2,2
"{'months': 11, 'years': 0}",3,0,0,3
"{'months': 12, 'years': 1}",4,1,13,18
"{'months': 2, 'years': 0}",3,0,2,5
"{'months': 24, 'years': 2}",3,1,4,8
"{'months': 3, 'years': 0}",3,1,1,5
"{'months': 36, 'years': 3}",0,2,2,4
"{'months': 4, 'years': 0}",3,0,3,6


In [91]:
print(df_groupby_date_label.index)
df_groupby_date_label.drop(df_groupby_date_label[df_groupby_date_label.index > 12].index, inplace=True)
df_groupby_date_label = df_groupby_date_label.sort_index(ascending=False)
# Since graph_objects automatically sort the index in ascending, we put it in string, so it doesn't sort
df_groupby_date_label.index = df_groupby_date_label.index.astype(str)
df_groupby_date_label

Index(['{'months': 1, 'years': 0}', '{'months': 10, 'years': 0}',
       '{'months': 11, 'years': 0}', '{'months': 12, 'years': 1}',
       '{'months': 2, 'years': 0}', '{'months': 24, 'years': 2}',
       '{'months': 3, 'years': 0}', '{'months': 36, 'years': 3}',
       '{'months': 4, 'years': 0}', '{'months': 48, 'years': 4}',
       '{'months': 5, 'years': 0}', '{'months': 6, 'years': 0}',
       '{'months': 60, 'years': 5}', '{'months': 7, 'years': 0}',
       '{'months': 72, 'years': 6}', '{'months': 8, 'years': 0}',
       '{'months': 84, 'years': 7}', '{'months': 9, 'years': 0}'],
      dtype='object', name='date')


TypeError: '>' not supported between instances of 'str' and 'int'

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Line(x=df_groupby_date_label.index,
                      y=df_groupby_date_label['POSITIVE']/df_groupby_date_label['TOTAL']*100,
                      line=dict(color='#00cc96'),
                      name='Positive',
                      text=['{:.2f}%'.format(v) for v in df_groupby_date_label['POSITIVE']/df_groupby_date_label['TOTAL']*100], hovertemplate='%{text}<extra></extra>'))

fig1.add_trace(go.Line(x=df_groupby_date_label.index,
                      y=df_groupby_date_label['NEGATIVE']/df_groupby_date_label['TOTAL']*100,
                      name='Negative',
                      line=dict(color='#eb533a'),
                      text=['{:.2f}%'.format(v) for v in df_groupby_date_label['NEGATIVE']/df_groupby_date_label['TOTAL']*100], hovertemplate='%{text}<extra></extra>'))

fig1.add_trace(go.Line(x=df_groupby_date_label.index,
                      y=df_groupby_date_label['NEUTRAL']/df_groupby_date_label['TOTAL']*100,
                      name='Neutral',
                      line=dict(color='#636efa'),
                      text=['{:.2f}%'.format(v) for v in df_groupby_date_label['NEUTRAL']/df_groupby_date_label['TOTAL']*100], hovertemplate='%{text}<extra></extra>'))

fig1.update_layout(title='Sentiment Analysis over Time', xaxis_title='Month(s) ago', yaxis_title='Percentage %', yaxis_range=[0,100])
fig1.show()

C:\Users\jacqu\anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [ ]:
#figs_json.append(pio.to_json(fig))

# Subgraph

In [ ]:
y_values = df_groupby_date_label['POSITIVE']/df_groupby_date_label['TOTAL']*100

x_numeric = np.arange(len(df_groupby_date_label.index)).reshape(-1, 1)

model = LinearRegression()
model.fit(x_numeric, y_values)
line = model.predict(x_numeric)

hover_text = ['Y: {:.2f}%<br>Percent Change with previous month: {:.2f}%<br>Absolute Change with previous month: {:.2f}%'.format(y_values[i], (y_values[i]-y_values[i-1])/y_values[i-1]*100, y_values[i]-y_values[i-1]) if y_values[i]-y_values[i-1] <= 0 else 'Y: {:.2f}%<br>Percent Change with previous month: +{:.2f}%<br>Absolute Change with previous month: +{:.2f}%'.format(y_values[i], (y_values[i]-y_values[i-1])/y_values[i-1]*100, y_values[i]-y_values[i-1]) for i in range(1, len(y_values))]
hover_text.insert(0, 'Y: '+"{:.2f}".format(y_values[0])+'%<br>Percent Change with previous month: 0%<br>Absolute Change with previous month: 0%')

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=df_groupby_date_label.index,
                         y=y_values,
                         mode='markers',
                         marker=dict(color='#00cc96'),
                         name='Positive',
                         text=hover_text,
                         hovertemplate='%{text}<extra></extra>'))

fig2.add_trace(go.Scatter(x=df_groupby_date_label.index,
                         y=line,
                         mode='lines',
                         line=dict(color='yellow'),
                         name='Regression Line',
                         text=['Percentage Change over the whole period: {:.2f}%'.format(line[-1]-line[0]) if (line[-1]-line[0]) <= 0 else 'Percentage Change over the whole period: +{:.2f}%'.format(line[-1]-line[0])  for i in range(len(y_values))],
                         hovertemplate='%{text}<extra></extra>'))

fig2.update_layout(title='Positive Change over Time', xaxis_title='Month(s) ago', yaxis_title='Percentage', yaxis_range=[0,100])
fig2.show()

# Subgraph

In [ ]:
y_values = df_groupby_date_label['NEGATIVE']/df_groupby_date_label['TOTAL']*100

x_numeric = np.arange(len(df_groupby_date_label.index)).reshape(-1, 1)

model = LinearRegression()
model.fit(x_numeric, y_values)
line = model.predict(x_numeric)

hover_text = ['Y: {:.2f}%<br>Percent Change with previous month: {:.2f}%<br>Absolute Change with previous month: {:.2f}%'.format(y_values[i], (y_values[i]-y_values[i-1])/y_values[i-1]*100, y_values[i]-y_values[i-1]) if y_values[i]-y_values[i-1] <= 0 else 'Y: {:.2f}%<br>Percent Change with previous month: +{:.2f}%<br>Absolute Change with previous month: +{:.2f}%'.format(y_values[i], (y_values[i]-y_values[i-1])/y_values[i-1]*100, y_values[i]-y_values[i-1]) for i in range(1, len(y_values))]
hover_text.insert(0, 'Y: '+"{:.2f}".format(y_values[0])+'%<br>Percent Change with previous month: 0%<br>Absolute Change with previous month: 0%')

fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=df_groupby_date_label.index,
                         y=y_values,
                         mode='markers',
                         marker=dict(color='#eb533a'),
                         name='Negative',
                         text=hover_text,
                         hovertemplate='%{text}<extra></extra>'))

fig3.add_trace(go.Scatter(x=df_groupby_date_label.index,
                         y=line,
                         mode='lines',
                         line=dict(color='yellow'),
                         name='Regression Line',
                         text=['Percentage Change over the whole period: {:.2f}%'.format(line[-1]-line[0]) if (line[-1]-line[0]) <= 0 else 'Percentage Change over the whole period: +{:.2f}%'.format(line[-1]-line[0]) for i in range(len(y_values))],
                         hovertemplate='%{text}<extra></extra>'))

fig3.update_layout(title='Negative Change over Time', xaxis_title='Month(s) ago', yaxis_title='Percentage', yaxis_range=[0,100])
fig3.show()

# Subgraph

In [ ]:
y_values = df_groupby_date_label['NEUTRAL']/df_groupby_date_label['TOTAL']*100

x_numeric = np.arange(len(df_groupby_date_label.index)).reshape(-1, 1)

model = LinearRegression()
model.fit(x_numeric, y_values)
line = model.predict(x_numeric)

hover_text = ['Y: {:.2f}%<br>Percent Change with previous month: {:.2f}%<br>Absolute Change with previous month: {:.2f}%'.format(y_values[i], (y_values[i]-y_values[i-1])/y_values[i-1]*100, y_values[i]-y_values[i-1]) if y_values[i]-y_values[i-1] <= 0 else 'Y: {:.2f}%<br>Percent Change with previous month: +{:.2f}%<br>Absolute Change with previous month: +{:.2f}%'.format(y_values[i], (y_values[i]-y_values[i-1])/y_values[i-1]*100, y_values[i]-y_values[i-1]) for i in range(1, len(y_values))]
hover_text.insert(0, 'Y: '+"{:.2f}".format(y_values[0])+'%<br>Percent Change with previous month: 0%<br>Absolute Change with previous month: 0%')

fig4 = go.Figure()
fig4.add_trace(go.Scatter(x=df_groupby_date_label.index,
                         y=y_values,
                         mode='markers',
                         marker=dict(color='#636efa'),
                         name='Neutral',
                         text=hover_text,
                         hovertemplate='%{text}<extra></extra>'))

fig4.add_trace(go.Scatter(x=df_groupby_date_label.index,
                         y=line,
                         mode='lines',
                         line=dict(color='yellow'),
                         name='Regression Line',
                         text=['Percentage Change over the whole period: {:.2f}%'.format(line[-1]-line[0]) if (line[-1]-line[0]) <= 0 else 'Percentage Change over the whole period: +{:.2f}%'.format(line[-1]-line[0]) for i in range(len(y_values))],
                         hovertemplate='%{text}<extra></extra>'))

fig4.update_layout(title='Neutral Change over Time', xaxis_title='Month(s) ago', yaxis_title='Percentage', yaxis_range=[0,100])
fig4.show()

# Graph

In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go

# Create subplots with 4 rows and 1 column
fig = sp.make_subplots(rows=4, cols=1)

# Add all figures
for trace in fig1.data:
    fig.add_trace(trace, row=1, col=1)
for trace in fig2.data:
    fig.add_trace(trace, row=2, col=1)
for trace in fig3.data:
    fig.add_trace(trace, row=3, col=1)
for trace in fig4.data:
    fig.add_trace(trace, row=4, col=1)

fig.update_yaxes(range=[0, 100])
fig.update_layout(title_text="Sentiment Analysis over Time", height=800)

fig.add_annotation(text="Month(s) ago", xref="paper", yref="paper", x=0.5, y=-0.1, showarrow=False)
fig.add_annotation(text="Percentage %", textangle=-90, xref="paper", yref="paper", x=-0.1, y=0.5, showarrow=False)

# Show the combined subplots
fig.show()

In [ ]:
figs_json.append(pio.to_json(fig))

# Graph Positif, neutre, négatif : diagramme circulaire (camembert)

In [ ]:
df_pie = df.groupby('label (roberta)').size()
print(df_pie)

label (roberta)
NEGATIVE    1319
NEUTRAL      441
POSITIVE    1949
dtype: int64


In [ ]:
fig = go.Figure(data=[go.Pie(labels=df_pie.index, values=df_pie.values)])

fig.show()

In [ ]:
figs_json.append(pio.to_json(fig))

# Graph MOYENNE ETOILES

In [ ]:
df_groupby_date_score = df.groupby(['date', 'grading']).size().unstack(fill_value=0)
df_groupby_date_score

grading,1,2,3,4,5
date,,,,,
0,13,3,0,2,6
1,36,12,13,14,64
2,29,6,5,16,44
3,33,10,6,16,45
4,43,5,10,10,47
5,36,5,8,7,29
6,35,4,9,16,39
7,46,2,7,16,37
8,31,8,6,12,33


In [ ]:
df_groupby_date_score['Mean'] = (np.array(df_groupby_date_score.columns[:5]) * df_groupby_date_score.values).sum(axis=1) / df_groupby_date_score.values.sum(axis=1)
df_groupby_date_score['Mean']

date
0      2.375000
1      3.417266
2      3.400000
3      3.272727
4      3.113043
5      2.858824
6      3.194175
7      2.962963
8      3.088889
9      3.090909
10     3.397959
11     2.822785
12     3.343612
24     2.957606
36     3.442222
48     3.394850
60     3.512329
72     3.374269
84     3.318182
96     3.250000
108    3.285714
120    3.500000
168    2.000000
Name: Mean, dtype: float64

In [ ]:
df_groupby_date_score.drop(df_groupby_date_score[df_groupby_date_score.index > 12].index, inplace=True)
df_groupby_date_score = df_groupby_date_score.sort_index(ascending=False)
# Since graph_objects automatically sort the index in ascending, we put it in string, so it doesn't sort
df_groupby_date_score.index = df_groupby_date_score.index.astype(str)
df_groupby_date_score

grading,1,2,3,4,5,Mean
date,,,,,,
12,216,33,28,109,295,3.343612
11,37,2,4,10,26,2.822785
10,29,5,7,12,45,3.397959
9,38,7,3,10,41,3.090909
8,31,8,6,12,33,3.088889
7,46,2,7,16,37,2.962963
6,35,4,9,16,39,3.194175
5,36,5,8,7,29,2.858824
4,43,5,10,10,47,3.113043


In [ ]:
fig = go.Figure()

fig.add_trace(go.Line(x=df_groupby_date_score.index,
                      y=df_groupby_date_score['Mean'],
                      name='Rating',
                      text=['{:.2f}'.format(v) for v in df_groupby_date_score['Mean']], hovertemplate='%{text}<extra></extra>'))

fig.update_layout(title='Star Rating over Time', xaxis_title='Month(s) ago', yaxis_title='Percentage', yaxis_range=[0,5])

fig.show()

C:\Users\jacqu\anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [ ]:
figs_json.append(pio.to_json(fig))

In [ ]:
figs_json

['{"data":[{"hovertemplate":"%{text}<extra></extra>","line":{"color":"#00cc96"},"name":"Positive","text":["54.79%","37.97%","58.16%","51.02%","40.00%","44.44%","53.40%","40.48%","46.09%","51.82%","53.00%","53.24%","29.17%"],"x":["12","11","10","9","8","7","6","5","4","3","2","1","0"],"y":[54.78645066273933,37.9746835443038,58.16326530612245,51.02040816326531,40.0,44.44444444444444,53.398058252427184,40.476190476190474,46.08695652173913,51.81818181818182,53.0,53.23741007194245,29.166666666666668],"type":"scatter","xaxis":"x","yaxis":"y"},{"hovertemplate":"%{text}<extra></extra>","line":{"color":"#eb533a"},"name":"Negative","text":["36.97%","49.37%","34.69%","43.88%","48.89%","42.59%","36.89%","46.43%","45.22%","35.45%","35.00%","37.41%","58.33%"],"x":["12","11","10","9","8","7","6","5","4","3","2","1","0"],"y":[36.96612665684831,49.36708860759494,34.69387755102041,43.87755102040816,48.888888888888886,42.592592592592595,36.89320388349515,46.42857142857143,45.21739130434783,35.45454545454